### chr done: 3,5,7,10,13,14,18,20,21,22

### chrs that have repeat SNP positions need to impute again

In [1]:
pos = read.table('../temp/PSA.autosome.bim')

In [2]:
bim = pos[c('V1','V4')]

In [3]:
dup = pos[duplicated(bim),]

In [5]:
pos[duplicated(bim),]

,V1,V2,V3,V4,V5,V6
56,1,rs1921,0,949608,A,G
3678,1,VG01S2022,0,11854476,G,T
53127,1,rs3817870,0,209849002,T,C
57623,1,rs1051741,0,226032229,T,C
83410,2,rs2576709,0,55476591,A,G
91565,2,rs7148,0,85133216,A,G
121277,2,rs33946522,0,219527712,T,C
199567,4,rs3926327,0,74284686,T,C
303866,6,rs780556,0,49802697,T,C
394718,8,rs2843740,0,37985897,A,G


In [4]:
write.table(dup$V2, 'repeat.snp.psa.txt', col.names = FALSE,row.names = FALSE,quote = FALSE)

In [1]:
!ls ../impute-mm3-psa/data/plink/*.bed

../impute-mm3-psa/data/plink/dataname_1kg_p1v3_10.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_13.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_14.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_18.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_20.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_21.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_22.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_3.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_5.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_7.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_qc_10.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_qc_13.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_qc_14.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_qc_18.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_qc_20.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_qc_21.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_qc_22.bed
../impute-mm3-psa/data/plink/dataname_1kg_p1v3_qc_3.bed
../imput

### chr 1,2,4,6,8,9,11,12,15,16,17,19

# remove repeat SNP

In [1]:
!head -n 10 repeat.snp.psa.txt

rs1921
VG01S2022
rs3817870
rs1051741
rs2576709
rs7148
rs33946522
rs3926327
rs780556
rs2843740


In [2]:
!plink --bfile ../temp/PSA.autosome --exclude repeat.snp.psa.txt --make-bed --out ../temp/psa-norepeat

PLINK v1.90b3.38 64-bit (7 Jun 2016)       https://www.cog-genomics.org/plink2
(C) 2005-2016 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ../temp/psa-norepeat.log.
Options in effect:
  --bfile ../temp/PSA.autosome
  --exclude repeat.snp.psa.txt
  --make-bed
  --out ../temp/psa-norepeat

128940 MB RAM detected; reserving 64470 MB for main workspace.
Allocated 27198 MB successfully, after larger attempt(s) failed.
790941 variants loaded from .bim file.
2847 people (1358 males, 1489 females) loaded from .fam.
2847 phenotype values loaded from .fam.
--exclude: 790922 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2847 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%

In [3]:
!grep '30228656' ../temp/PSA.autosome.bim

17	GA032273	0	30228656	A	G
17	rs2074101	0	30228656	A	G


In [4]:
!grep -w '30228656' ../temp/psa-norepeat.bim

17	GA032273	0	30228656	A	G


In [4]:
# %load ../Tools/impute-mm3/init.sh
!mkdir data job_reports
!mkdir data/haplotypes data/imputed data/plink data/target
!mkdir job_reports/shapeit

/vlsci/SG0003/wei_lu/imputation/impute-mm3-slurm-chr22
/vlsci/SG0003/wei_lu/imputation


# step 0 enviroment setting

In [8]:
%%writefile parameters.sh
#!/usr/bin/env bash

# Exe
######################
readonly shapeit2="shapeit"
readonly mm3omp="Minimac3-omp"
readonly plink2="plink"
readonly genetdistR="${wd}genetdist.R"

# Initial QC thresholds
######################
MAF="0.005"
HWE="0.000001"
geno="0.05"
mind="0.05"

# Filtering thresholds
######################
filterMAF="0.000001"
filterHWE="0.000001"
filtergeno="0.02"
filtermind="0.1"
filterInfo="0.3"

#Change according to data set
#############################
rawdata="${wd}data/target/dataname" 
impdata="dataname_1kg_p1v3"

#Input/Output Data
##################
inputdata="${wd}../temp/psa-norepeat"
targetdata="${wd}data/target/DAT${chr}"
targetdatadir="${wd}data/target/"
hapdatadir="${wd}data/haplotypes/"
hapout="${hapdatadir}DAT${chr}"
targetvcf="${hapdatadir}DAT${chr}.vcf"
impdatadir="${wd}data/imputed/"
plinkimpdir="${wd}data/plink/"
imputedvcf="${wd}data/imputed/data_1kg_p1v3_${chr}"
imputedplink="${wd}data/plink/${impdata}_${chr}"
imputedplinkqc="${wd}data/plink/${impdata}_qc_${chr}"
chrdata="DAT${chr}"

#Reference Data for Phasing
###########################
refdatadir="/vlsci/SG0003/shared/imputation/reference/1000GP_Phase3/"

reflegend="${refdatadir}1000GP_Phase3_chr${chr}.legend.gz"
refhaps="${refdatadir}1000GP_Phase3_chr${chr}.hap.gz"
refsample="${refdatadir}ALL_1000G_phase1integrated_v3.sample"
refgmap="${refdatadir}genetic_map_chr${chr}_combined_b37.txt"
refsample="${refdatadir}1000GP_Phase3.sample"
#reflegend="${refdatadir}ALL_1000G_phase1integrated_v3_chr${chr}.legend.gz"
#refhaps="${refdatadir}ALL_1000G_phase1integrated_v3_chr${chr}.hap.gz"
#refsample="${refdatadir}ALL_1000G_phase1integrated_v3.sample"
#refgmap="${refdatadir}genetic_map_chr${chr}_combined_b37.txt"

#Reference Data for Imputation - m3vcf Phase1
#############################################
refdatadir="/vlsci/SG0003/shared/imputation/reference/"
refvcf="${refdatadir}${chr}.1000g.Phase3.v5.With.Parameter.Estimates.m3vcf.gz"

# shapeit2 (use reference set if less than 50 individuals)
#########################################################
minindiv=50

# minimac3 interval (default is 5Mb) (currently not used)
#########################################################
interval=5000000

# Additional directories required
####################################

mkdir -p ${targetdatadir}
mkdir -p ${hapdatadir}
mkdir -p $impdatadir
mkdir -p $plinkimpdir


Overwriting parameters.sh


# calculate genetic distance
map.sh (Extract by chr and fill in genetic distance from map file

In [4]:
%%writefile map.sh
#!/bin/bash
#SBATCH --time=00:30:00
#SBATCH --mem-per-cpu=8000
#SBATCH --array=1,2,4,6,8,9,11,12,15,16,17,19%6
#SBATCH -p sysgen

# This script will take a binary plink file and:

# 1. extract individual chromosomes
# 2. update genetic distance from map file
# (For the SNPs that don't have a genetic position, SHAPEIT internally
# determines its genetic position using linear interpolation)

# set -e
if [ -n "${1}" ]; then
    SLURM_ARRAY_TASK_ID=${1}
fi

chr=${SLURM_ARRAY_TASK_ID}

wd=`pwd`"/"

source parameters.sh

${plink2} --bfile ${inputdata} --chr $chr --make-bed --out ${targetdata} 
R --no-save --args ${targetdata}.bim ${refgmap} < ${genetdistR}
${plink2} --noweb --bfile ${targetdata} --make-bed --out ${targetdata} 


Overwriting map.sh


In [7]:
!sbatch map.sh

Submitted batch job 2080164


In [15]:
!sacct -j 2080164 --format=JobID,JobName,ReqMem,MaxRSS,Elapsed,State,TotalCPU

       JobID    JobName     ReqMem     MaxRSS    Elapsed      State   TotalCPU 
------------ ---------- ---------- ---------- ---------- ---------- ---------- 
2080164_19       map.sh     8000Mc              00:00:03  COMPLETED  00:00.753 
2080164_19.+      batch     8000Mc          0   00:00:03  COMPLETED  00:00.753 
2080164_19.+     extern     8000Mc          0   00:00:06  COMPLETED   00:00:00 
2080164_1        map.sh     8000Mc              00:00:06  COMPLETED  00:02.596 
2080164_1.b+      batch     8000Mc          0   00:00:06  COMPLETED  00:02.596 
2080164_1.e+     extern     8000Mc          0   00:00:09  COMPLETED   00:00:00 
2080164_2        map.sh     8000Mc              00:00:05  COMPLETED  00:02.997 
2080164_2.b+      batch     8000Mc          0   00:00:05  COMPLETED  00:02.997 
2080164_2.e+     extern     8000Mc          0   00:00:08  COMPLETED   00:00:00 
2080164_4        map.sh     8000Mc              00:00:04  COMPLETED  00:01.998 
2080164_4.b+      batch     8000Mc      

# step 2 shapeit
hap.sh (Performing phasing using shapeit2, using reference set if < 50 individuals)

In [6]:
%%writefile hap.sh
#!/bin/bash
#SBATCH --time=03:00:00
#SBATCH --mem-per-cpu=1000
#SBATCH --job-name=job_reports/phase
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=16
#SBATCH --array=1,2,4,6,8,9,11,12,15,16,17,19
#SBATCH -p sysgen
#SBATCH --mail-type=END  
#SBATCH --mail-user=l5@student.unimelb.edu.au

# This script will take a reference-aligned binary plink file and:
# 1. For each chromosome perform shapeit phasing
# 2. Remove snps that don't align between reference and main panel
# 3. Convert to vcf format

# set -e
if [ -n "${1}" ]; then
    SLURM_ARRAY_TASK_ID=${1}
fi

chr=${SLURM_ARRAY_TASK_ID}
wd=$(pwd)"/"
echo $wd
source $wd/parameters.sh
threads=16

flags="--thread $threads --noped"

#if [ "${chr}" -eq "23" ]; then
#    flags="$flags --chrX"
#fi

#if [[ -f ${targetdata}"-align.bim" ]];
#then
#    targetdata="${targetdata}-align"
#    echo "Found aligned set"
#fi

#indv=$(wc -l ${targetdata}.fam | cut -f 1 -d ' ')
#echo "There are $indv individuals"

echo $SLURM_ARRAY_TASK_ID

bed="../temp/psa-norepeat"
plink --bfile $bed --chr $chr --make-bed --out 'data/target/DAT'${chr}
targetdata="data/target/DAT${chr}"

## If less than 100 indivduals, use reference panel to phase
minindiv=100
if [ $indv -lt $minindiv ]; then
    flags="$flags --input-ref ${refhaps} ${reflegend} ${refsample}"
    echo "Using reference set to phase data"
fi

## Performing phasing 
${shapeit2} --input-bed ${targetdata}.bed ${targetdata}.bim ${targetdata}.fam \
            --input-map ${refgmap} \
            --output-max ${hapout}.haps ${hapout}.sample ${flags} \
            --output-log ${wd}job_reports/shapeit.$chr

## Convert data to vcf for minimac3
${shapeit2} -convert \
            --input-haps ${hapout} \
            --output-vcf ${targetvcf} \
            --output-log ${wd}job_reports/shapeit.$chr \
            --thread $threads


Overwriting hap.sh


In [7]:
!sbatch hap.sh

Submitted batch job 2080285


In [20]:
!shapeit --input-bed data/target/DAT17.bed data/target/DAT17.bim data/target/DAT17.fam \
            --input-map /vlsci/SG0003/shared/imputation/reference/1000GP_Phase3/genetic_map_chr17_combined_b37.txt \
            --output-max data/haplotypes/17.haps data/haplotypes/17.sample --thread 2 --noped \
            --output-log job_reports/shapeit.chr17


Segmented HAPlotype Estimation & Imputation Tool
  * Authors : Olivier Delaneau, Jared O'Connell, Jean-FranÃ§ois Zagury, Jonathan Marchini
  * Contact : send an email to the OXSTATGEN mail list https://www.jiscmail.ac.uk/cgi-bin/webadmin?A0=OXSTATGEN
  * Webpage : https://mathgen.stats.ox.ac.uk/shapeit
  * Version : v2.r837
  * Date    : 08/02/2017 22:56:08
  * LOGfile : [job_reports/shapeit.chr17.log]

MODE -phase : PHASING GENOTYPE DATA
  * Autosome (chr1 ... chr22)
  * Window-based model (SHAPEIT v2)
  * MCMC iteration

Parameters :
  * Seed : 1486554968
  * Parallelisation: 2 threads
  * Ref allele is NOT aligned on the reference genome
  * MCMC: 35 iterations [7 B + 1 runs of 8 P + 20 M]
  * Model: 100 states per window [100 H + 0 PM + 0 R + 0 COV ] / Windows of ~2.0 Mb / Ne = 15000
  * Pedigree information not taken into account

Reading site list in [data/target/DAT17.bim]

ERROR: Duplicate site pos=30228656 ref=A alt=G


In [10]:
!sacct -j 2080285 --format=JobID,JobName,ReqMem,MaxRSS,Elapsed,State,TotalCPU

       JobID    JobName     ReqMem     MaxRSS    Elapsed      State   TotalCPU 
------------ ---------- ---------- ---------- ---------- ---------- ---------- 
2080285_19   job_repor+     1000Mc              00:30:41  COMPLETED   07:30:31 
2080285_19.+      batch     1000Mc   1360548K   00:30:41  COMPLETED   07:30:31 
2080285_19.+     extern     1000Mc          0   00:30:43  COMPLETED   00:00:00 
2080285_1    job_repor+     1000Mc              02:18:10  COMPLETED 1-09:47:04 
2080285_1.b+      batch     1000Mc   5040720K   02:18:10  COMPLETED 1-09:47:04 
2080285_1.e+     extern     1000Mc          0   02:18:11  COMPLETED   00:00:00 
2080285_2    job_repor+     1000Mc              02:12:48  COMPLETED 1-08:23:28 
2080285_2.b+      batch     1000Mc   4829880K   02:12:48  COMPLETED 1-08:23:28 
2080285_2.e+     extern     1000Mc          0   02:12:49  COMPLETED   00:00:00 
2080285_4    job_repor+     1000Mc              01:33:04  COMPLETED   22:46:00 
2080285_4.b+      batch     1000Mc   367

# step 3 
impute.sh (Perform imputation using minimac3, convert vcf output to plink2 and do post imputation qc)

In [8]:
%%writefile impute.sh
#!/bin/bash
#SBATCH --time=8:00:00
#SBATCH --mem-per-cpu=12000
#SBATCH --job-name=imputemm3
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=16
#SBATCH --array=1,2,4,6,8,9,11,12,15,16,17,19
#SBATCH -p sysgen
#SBATCH --mail-type=END  
#SBATCH --mail-user=l5@student.unimelb.edu.au


module load zlib/1.2.8-GCC-4.9.3
module load glibc/2.15-GCC-4.9.3

if [ -n "${1}" ]; then
    SLURM_ARRAY_TASK_ID=${1}
fi

chr=${SLURM_ARRAY_TASK_ID}

wd=`pwd`"/"
source parameters.sh
threads=16

#echo "Imputing on $threads threads"

#impute
${mm3omp} --refHaps $refvcf \
          --haps $targetvcf \
          --prefix $imputedvcf \
          --cpus $threads

#convert to plink format
plink --vcf ${imputedvcf}.dose.vcf.gz --double-id --make-bed --out $imputedplink --threads $threads
#quality control 
plink --bfile ${imputedplink} --hwe ${filterHWE} --maf ${filterMAF} --make-bed --out ${imputedplinkqc} \
--threads $threads


Overwriting impute.sh


In [ ]:
!sbatch impute.sh

In [14]:
!sacct -j 2080339 --format=JobID,JobName,ReqMem,AveRSS,Elapsed,State,TotalCPU

       JobID    JobName     ReqMem     AveRSS    Elapsed      State   TotalCPU 
------------ ---------- ---------- ---------- ---------- ---------- ---------- 
2080339_19    imputemm3    12000Mc              01:59:46  COMPLETED   05:36:58 
2080339_19.+      batch    12000Mc      3240K   01:59:46  COMPLETED   05:36:58 
2080339_19.+     extern    12000Mc          0   01:59:47  COMPLETED   00:00:00 
2080339_1     imputemm3    12000Mc              06:45:22  COMPLETED   18:42:14 
2080339_1.b+      batch    12000Mc      3236K   06:45:22  COMPLETED   18:42:14 
2080339_1.e+     extern    12000Mc          0   06:45:25  COMPLETED   00:00:00 
2080339_2     imputemm3    12000Mc              07:21:24  COMPLETED   20:22:43 
2080339_2.b+      batch    12000Mc      3248K   07:21:24  COMPLETED   20:22:43 
2080339_2.e+     extern    12000Mc          0   07:21:25  COMPLETED   00:00:00 
2080339_4     imputemm3    12000Mc              05:54:20  COMPLETED   16:21:12 
2080339_4.b+      batch    12000Mc      

In [1]:
!wc -l impute-mm3-slurm-chr22/data/plink/dataname_1kg_p1v3_22.bim
!wc -l impute-mm3-slurm-chr22/data/plink/dataname_1kg_p1v3_qc_22.bim

652195 impute-mm3-slurm-chr22/data/plink/dataname_1kg_p1v3_22.bim
285381 impute-mm3-slurm-chr22/data/plink/dataname_1kg_p1v3_qc_22.bim


In [2]:
!wc -l impute-mm3-slurm-chr22/data/imputed/data_1kg_p1v3_22.info

652196 impute-mm3-slurm-chr22/data/imputed/data_1kg_p1v3_22.info


In [24]:
!wc -l data/psa-chr22.bim

13276 data/psa-chr22.bim
